Install packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import json

Access tibia.com and populate a list with creature names and the respective URL

In [409]:
creatures_list = []
url_monsters =  "https://www.tibia.com/library/?subtopic=creatures"

raw_html = requests.get(url_monsters)
bs1 = BeautifulSoup(raw_html.text, "lxml")

bs1_parse1 = bs1.find("div", {"class": "Creatures"})
bs1_parse2 = bs1_parse1.find_all("div")

for div in (bs1_parse2):
    if div.find("a"): creatures_list.append({"name":div.text.rstrip().lstrip(),"url":div.find("a")['href']})

Populates creature entities with more information

In [410]:
i = 1

for creature in creatures_list:
    raw_html2 = requests.get(creature["url"])
    bs2 = BeautifulSoup(raw_html2.text, "lxml")

    bs2_parse1 = bs2.find_all("p")

    # Creates the indexes that will be populated
    creature["hit_points"] = 0
    creature["immune_to"] = None
    creature["strong_vs"] = None
    creature["weak_vs"] = None
    creature ["can_be_summoned"] = 0
    creature["can_be_convinced"] = 0
    creature["see_invisible"] = 0
    creature ["can_be_paralysed"] = 0
    creature["exp_points"] = 0
    del creature["url"]

    ## Gets "hit_points"
    hp_phrase = bs2_parse1[1].text.split(".")[0]
    creature["hit_points"] = hp_phrase[hp_phrase.find("have") + 5 : hp_phrase.find("hitpoints") - 1]

    ## Gets "immune_to", "strong_vs" and "weak_vs"
    for new_info in bs2_parse1[1].text.split(".")[1:-1]:
        if new_info.find("immune") != -1:
            creature["immune_to"] = new_info[new_info.find("immune to") + 10 : new_info.find("damage")-1] 
        elif new_info.find("strong") != -1:
            creature["strong_vs"] = new_info[new_info.find("strong against") + 15 : new_info.find("damage")-1]
        elif new_info.find("weak") != -1:
            creature["weak_vs"] = new_info[new_info.find("weak against") + 13 : new_info.find("damage")-1]

    # Gets "can_be_summoned" and "can_be_convinced":
    if bs2_parse1[1].text.find("neither be summoned nor convinced") != -1:
        pass
    elif bs2_parse1[1].text.find("but they cannot be summoned") != -1:
        creature["can_be_summoned"] = 0
        creature["can_be_convinced"] = 1
    elif bs2_parse1[1].text.find("to summon or convince") != -1:
        creature["can_be_summoned"] = 1
        creature["can_be_convinced"] = 1
    else:
        creature["can_be_summoned"] = 1
        creature["can_be_convinced"] = 0
    
    ## Gets "see_invisible"
    if bs2_parse1[1].text.find("able to sense invisible creatures") != -1: creature["see_invisible"] = 1
    
    ## Gets "can_be_paralysed"
    if bs2_parse1[1].text.find("cannot be paralysed") == -1: creature["can_be_paralysed"] = 1
    
    ## Gets "exp_points"
    xp_phrase = bs2_parse1[2].text.split(".")[0]
    creature["exp_points"] = xp_phrase[xp_phrase.find("yield") + 6 : xp_phrase.find("experience") - 1]

    
    print("----------")
    print("Creatures {} out of {} parsed.".format(i,len(creatures_list)))
    print(creature)

    i += 1
    sleep (1) ## waits 3 secs not to overload tibia.com with requests

print(" -- end -- ")

----------
Creatures 1 out of 618 parsed.
{'name': 'Acid Blobs', 'hit_points': '250', 'immune_to': 'death and earth', 'strong_vs': 'ice', 'weak_vs': 'energy and fire', 'can_be_summoned': 0, 'can_be_convinced': 0, 'see_invisible': 0, 'can_be_paralysed': 1, 'exp_points': '250'}
----------
Creatures 2 out of 618 parsed.
{'name': 'Acolytes Of The Cult', 'hit_points': '390', 'immune_to': None, 'strong_vs': 'earth, holy and ice', 'weak_vs': 'death, energy and physical', 'can_be_summoned': 0, 'can_be_convinced': 0, 'see_invisible': 1, 'can_be_paralysed': 1, 'exp_points': '300'}
----------
Creatures 3 out of 618 parsed.
{'name': 'Adepts Of The Cult', 'hit_points': '430', 'immune_to': None, 'strong_vs': 'earth, holy and ice', 'weak_vs': 'death and energy', 'can_be_summoned': 0, 'can_be_convinced': 0, 'see_invisible': 1, 'can_be_paralysed': 1, 'exp_points': '400'}
----------
Creatures 4 out of 618 parsed.
{'name': 'Adult Goannas', 'hit_points': '8300', 'immune_to': None, 'strong_vs': 'earth', 'w

Saving the creatures list in a JSON file

In [411]:

with open("creatures_data.json", "w", encoding = 'utf-8') as file:
    json.dump(creatures_list, file, ensure_ascii = 'False', indent = 2)

print("Data stored successfully")

Data stored successfully


Loads the creatures in a dataframe

In [368]:
creatures_df = pd.read_json("creatures_data.json")
creatures_df.head(2)

,name,hit_points,immune_to,strong_vs,weak_vs,can_be_summoned,can_be_convinced,see_invisible,can_be_paralysed,exp_points,week_vs
0,Acid Blobs,250,death and earth,ice,energy and fire,0,0,0,1,250,NaN
1,Acolytes Of The Cult,390,None,"earth, holy and ice","death, energy and physical",0,0,1,1,300,NaN


We'll manually create 1/0 flags for weak/strong/immune

In [412]:
elements = []

for creature in creatures_list:
    if creature["weak_vs"] != None:
        for element in creature["weak_vs"].replace(" and",",").split(","):
            elements.append(element.lstrip())
    if creature["strong_vs"] != None:
        for element in creature["strong_vs"].replace(" and",",").split(","):
            elements.append(element.lstrip())
    if creature["immune_to"] != None:
        for element in creature["immune_to"].replace(" and",",").split(","):
            elements.append(element.lstrip())    

elements_set = set(elements)

for element in elements_set:
    for creature in creatures_list:
        if creature["weak_vs"] != None:
            if creature["weak_vs"].find(element) != -1:
                creature["weak_vs_" + element] = 1
            else:
                creature["weak_vs_" + element] = 0
        else:
            creature["weak_vs_" + element] = 0

        if creature["strong_vs"] != None:
            if creature["strong_vs"].find(element) != -1:
                creature["strong_vs_" + element] = 1
            else:
                creature["strong_vs_" + element] = 0
        else:
            creature["strong_vs_" + element] = 0
        
        if creature["immune_to"] != None:
            if creature["immune_to"].find(element) != -1:
                creature["immune_to_" + element] = 1
            else:
                creature["immune_to_" + element] = 0
        else:
            creature["immune_to_" + element] = 0

In [ ]:
for creature in creatures_list:
    del creature["weak_vs"]
    del creature["strong_vs"]
    del creature["immune_to"]

Updated JSON File

In [420]:

with open("creatures_data_v2.json", "w", encoding = 'utf-8') as file:
    json.dump(creatures_list, file, ensure_ascii = 'False', indent = 2)

print("Data stored successfully")

Data stored successfully


Updated Dataframe

In [421]:
creatures_df = pd.read_json("creatures_data_v2.json")
creatures_df.head(2)

,name,hit_points,can_be_summoned,can_be_convinced,see_invisible,can_be_paralysed,exp_points,weak_vs_earth,strong_vs_earth,immune_to_earth,...,immune_to_holy,weak_vs_agony,strong_vs_agony,immune_to_agony,weak_vs_fire,strong_vs_fire,immune_to_fire,weak_vs_ice,strong_vs_ice,immune_to_ice
0,Acid Blobs,250,0,0,0,1,250,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,Acolytes Of The Cult,390,0,0,1,1,300,0,1,0,...,0,0,0,0,0,0,0,0,1,0
